%% Machine Learning Online Class - Exercise 3 | Part 1: One-vs-all
%  Instructions
%  ------------
%  This file contains code that helps you get started on the
%  linear exercise. You will need to complete the following functions
%  in this exericse:
%     lrCostFunction.m (logistic regression cost function)
%     oneVsAll.m
%     predictOneVsAll.m
%     predict.m
%  For this exercise, you will not need to change any code in this file,
%  or any other files other than those mentioned above.

In [1]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
from scipy.io import loadmat  
import seaborn as sbs
import matplotlib.image as mpimg
import scipy.optimize as op
%matplotlib inline

In [2]:
#%% Setup the parameters you will use for this part of the exercise
input_layer_size  = 400;  #% 20x20 Input Images of Digits
num_labels = 10;          #% 10 labels, from 1 to 10
                          #% (note that we have mapped "0" to label 10)

%% =========== Part 1: Loading and Visualizing Data =============
%  We start the exercise by first loading and visualizing the dataset.
%  You will be working with a dataset that contains handwritten digits.

In [3]:
#% Load Training Data
data = loadmat('ex3data1.mat');#% training data stored in arrays X, y
data['X'].shape
data['y'].shape

(5000L, 1L)

In [ ]:
m,n = data['X'].shape
#% Randomly select 100 data points to display
rand_indices = np.random.permutation(m)
sel = data['X'][rand_indices[0:100], :];
sel.shape

In [ ]:
def displayData(sel):
    m,n = sel.shape
    example_width = int(np.sqrt(n))
    example_height = int(n/example_width)
    display_rows = int(np.sqrt(m))
    display_cols = int(np.sqrt(m))
    pad = 1;
    display_array = np.ones((pad + display_rows * (example_height + pad),  pad + display_cols * (example_width + pad)))
    curr_ex = 0;
    for j in range(display_rows):
        for i in range(display_cols):
            #% Copy the patch
            #% Get the max value of the patch
            #max_val = max(abs(X(curr_ex, :)));
            array_temp = (sel[curr_ex,:].reshape((example_height,example_width))).T;
            u1=pad + j * (example_height + pad);
            u2=pad + j * (example_height + pad)+example_height;
            v1=pad + i * (example_width + pad);
            v2=pad + i * (example_width + pad)+example_width;
            display_array[u1:u2,v1:v2] = array_temp
            curr_ex = curr_ex + 1;
    return display_array

In [ ]:
display_array = displayData(sel)

In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(display_array)

%% ============ Part 2a: Vectorize Logistic Regression ============
%  In this part of the exercise, you will reuse your logistic regression
%  code from the last exercise. You task here is to make sure that your
%  regularized logistic regression implementation is vectorized. After
%  that, you will implement one-vs-all classification for the handwritten
%  digit dataset.

In [13]:
def sigmoid(z):
    #%SIGMOID Compute sigmoid function
    #%g = SIGMOID(z) computes the sigmoid of z.

    #% You need to return the following variables correctly 
    g = np.matrix(np.zeros(np.shape(z)));

#% ====================== YOUR CODE HERE ======================
#% Instructions: Compute the sigmoid of each value of z (z can be a matrix,vector or scalar).
    g = np.divide(1,(1+np.exp(-z)));    
    return g

def lrCostFunction(theta, X, y, lamb):
    #%COSTFUNCTIONREG Compute cost and gradient for logistic regression with regularization
    #%   J = COSTFUNCTIONREG(theta, X, y, lambda) computes the cost of using
    #%   theta as the parameter for regularized logistic regression and the
    #%   gradient of the cost w.r.t. to the parameters. 
    #% Initialize some useful values
    m=len(y);
    J = 0;
    grad = np.zeros(theta.shape);
    n = theta.shape[0];
    if len(np.shape(theta))==1:
        theta = np.transpose(np.matrix(theta)); 
    #Compute the cost of a particular choice of theta.
    #You should set J to the cost.
    #Compute the partial derivatives and set grad to the partial 
    #derivatives of the cost w.r.t. each parameter in theta
    H=sigmoid(X*theta);
    #J = sum((-y.*log(H)-(1-y).*log(1-H)))/m+lambda/2/m*sum(theta(2:n).^2);
    J = np.sum(np.multiply(-y,np.log(H))-np.multiply((1-y),np.log(1-H)))/m \
        +np.sum(np.power(theta[1:n],2))*lamb/2.0/m;
    grad=np.transpose(X)*(H-y)/m+lamb/m*theta;
    grad[0] = grad[0]-lamb/m*theta[0];
    #grad=1
    return J,grad

In [ ]:
theta_t = np.matrix([-2, -1, 1, 2]).T;
X_t = np.matrix(np.ones((5,1)))
tt = ((np.matrix(range(15)).reshape((3,5))).T+1.0)/10.0
X_t = np.append(X_t,tt,axis=1)
y_t = np.matrix([1,0,1,0,1]).T
       
lambda_t = 3.0;
J,grad = lrCostFunction(theta_t, X_t, y_t, lambda_t);
print('\nCost: %f' %J);
print('Expected cost: 2.534819');
print('Gradients:');
for i in range(len(grad)): print 'grad=%.4f  ' %(grad[i])
print('Expected gradients:\n');
print(' 0.146561\n -0.548558\n 0.724722\n 1.398003');
print('Program paused. Press enter to continue.');

In [116]:
def oneVsAll(X, y, num_labels, lamb):
#%ONEVSALL trains multiple logistic regression classifiers and returns all
#%the classifiers in a matrix all_theta, where the i-th row of all_theta 
#%corresponds to the classifier for label i
#%   [all_theta] = ONEVSALL(X, y, num_labels, lambda) trains num_labels
#%   logistic regression classifiers and returns each of these classifiers
#%   in a matrix all_theta, where the i-th row of all_theta corresponds 
#%   to the classifier for label i
#% Some useful variables
    m,n = X.shape

#% You need to return the following variables correctly 
    all_theta = np.zeros((num_labels, n + 1));

#% Add ones to the X data matrix
    X = np.c_[np.ones(m), X];
    optimal_theta = np.zeros((num_labels,n+1))
#% ====================== YOUR CODE HERE ======================
#% Instructions: You should complete the following code to train num_labels
#%               logistic regression classifiers with regularization
#%               parameter lambda. 
#%
#% Hint: theta[:] will return a column vector.
#% Hint: You can use y == c to obtain a vector of 1's and 0's that tell you
#%       whether the ground truth is true/false for this class.
#% Note: For this assignment, we recommend using fmincg to optimize the cost
#%       function. It is okay to use a for-loop (for c = 1:num_labels) to
#%       loop over the different classes.
#%
#%       fmincg works similarly to fminunc, but is more efficient when we
#%       are dealing with large number of parameters.

    for c in range(num_labels):  
         #% Set Initial theta
        initial_theta = np.zeros((n + 1, 1)); 
        yy = np.matrix(y==c+1)*1
         #% Set options for fminunc
        #options = optimset('GradObj', 'on', 'MaxIter', 50);
        Result = op.minimize(fun = lrCostFunction, 
                    x0 = initial_theta, 
                    args = (X, yy,lamb),
                    method = 'TNC',
                    jac = True);
        optimal_theta[c,:] = np.ravel(np.transpose(np.matrix(Result.x)))
    return optimal_theta      

In [117]:
#%% ============ Part 2b: One-vs-All Training ============
print('Training One-vs-All Logistic Regression...')
X=data['X']
y=data['y']
lamb = 0;
all_theta = oneVsAll(X, y, num_labels, lamb);
print('Done.')

Training One-vs-All Logistic Regression...
Done.


In [118]:
def predictOneVsAll(all_theta,X):
    #%PREDICT Predict the label for a trained one-vs-all classifier. The labels 
    #%are in the range 1..K, where K = size(all_theta, 1). 
    #%  p = PREDICTONEVSALL(all_theta, X) will return a vector of predictions
    #%  for each example in the matrix X. Note that X contains the examples in
    #%  rows. all_theta is a matrix where the i-th row is a trained logistic
    #%  regression theta vector for the i-th class. You should set p to a vector
    #%  of values from 1..K (e.g., p = [1; 3; 1; 2] predicts classes 1, 3, 1, 2
    #%  for 4 examples) 
    m,n = X.shape
    num_labels,nn = all_theta.shape;
    p = np.zeros((m, 1));
    X = np.c_[np.ones(m), X];
    #% Hint: This code can be done all vectorized using the max function.
#%       In particular, the max function can also return the index of the 
#%       max element, for more information see 'help max'. If your examples 
#%       are in rows, then, you can use max(A, [], 2) to obtain the max 
#%       for each row.
       
    A=sigmoid(X*(np.matrix(all_theta.T)))
    p=np.argmax(A,1)+1;
    return p
    

In [119]:
#%% ================ Part 3: Predict for One-Vs-All ================
pred = predictOneVsAll(all_theta,X);
accuracy = np.mean((pred==y)*1)*100
print('Training Set Accuracy: %f'% accuracy);

Training Set Accuracy: 97.600000


In [122]:
all_theta

array([[ -8.63159614e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.83430039e-02,   2.85731475e-07,   0.00000000e+00],
       [ -5.71590195e+00,   0.00000000e+00,   0.00000000e+00, ...,
          6.55275806e-02,  -6.83928011e-03,   0.00000000e+00],
       [ -8.66581914e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -2.72550208e-04,  -1.16072803e-06,   0.00000000e+00],
       ..., 
       [ -1.29089662e+01,   0.00000000e+00,   0.00000000e+00, ...,
         -6.81939212e+00,   7.87510982e-01,   0.00000000e+00],
       [ -8.61088375e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -1.99280663e-01,   1.08948233e-02,   0.00000000e+00],
       [ -9.28206159e+00,   0.00000000e+00,   0.00000000e+00, ...,
          9.00830871e-04,   4.01741201e-05,   0.00000000e+00]])